# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**: PabloQuinteroCorona

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Lab03PabloQuintero") \
    .master("spark://f5f576f1e425:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/22 05:00:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from PabloQuintero.spark_utils import SparkUtils
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

In [4]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/directory")

df_airlines.show(n=5)
#Aqui te muestra como esta el csv compuesto a la hora de correrlo

[Stage 0:>                                                          (0 + 1) / 1]

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

In [5]:
df_clean = df_airlines.select("airline", "source_city", "destination_city", 
                              "departure_time", "arrival_time", "stops", 
                              "duration", "price")
print("Columnas borrada")
df_clean.printSchema()
#Te muestra las celda que no necesitas y te muestra su esquema

Columnas borrada
root
 |-- airline: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- price: integer (nullable = true)



In [6]:
#Revisa los valores nulos
from pyspark.sql.functions import col, sum, when, col, concat, lit
df_clean.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_clean.columns]).show()



[Stage 1:=============================>                             (1 + 1) / 2]

+-------+-----------+----------------+--------------+------------+-----+--------+-----+
|airline|source_city|destination_city|departure_time|arrival_time|stops|duration|price|
+-------+-----------+----------------+--------------+------------+-----+--------+-----+
|      0|          0|               0|             0|           0|    0|       0|    0|
+-------+-----------+----------------+--------------+------------+-----+--------+-----+



In [7]:
#Columa stop
df_clean = df_clean.withColumn(
    "stops_num",
    when(col("stops") == "zero", 0)
    .when(col("stops") == "one", 1)
    .when(col("stops") == "two", 2)
    .when(col("stops") == "three", 3)
    .otherwise(None)
)

print("stops normalizado")
df_clean.select("stops", "stops_num").distinct().show()

stops normalizado


[Stage 4:=============================>                             (1 + 1) / 2]

+-----------+---------+
|      stops|stops_num|
+-----------+---------+
|       zero|        0|
|        one|        1|
|two_or_more|     NULL|
+-----------+---------+



In [8]:
df_clean = df_clean.withColumn(
    "route",
    concat(col("source_city"), lit(" → "), col("destination_city"))
)
df_clean.select("route").show(5)

+--------------+
|         route|
+--------------+
|Delhi → Mumbai|
|Delhi → Mumbai|
|Delhi → Mumbai|
|Delhi → Mumbai|
|Delhi → Mumbai|
+--------------+
only showing top 5 rows


In [12]:
df_clean = df_clean.withColumn(
    "departure_gate",
    when(col("departure_time").contains("AM"), 1)  # mañana
    .when(col("departure_time").contains("PM"), 2)  # tarde/noche
    .otherwise(0)  # early morning (si no detecta)
)

df_clean = df_clean.withColumn(
    "arrival_hub",
    when(col("arrival_time").contains("AM"), 1)
    .when(col("arrival_time").contains("PM"), 2)
    .otherwise(0)
)
df_clean.select("departure_time","arrival_time").show(5)


+--------------+-------------+
|departure_time| arrival_time|
+--------------+-------------+
|       Evening|        Night|
| Early_Morning|      Morning|
| Early_Morning|Early_Morning|
|       Morning|    Afternoon|
|       Morning|      Morning|
+--------------+-------------+
only showing top 5 rows


In [13]:
df_clean = df_clean.withColumn(
    "is_expensive",
    when(col("price") > 6000, True).otherwise(False)
)

df_clean.select("price", "is_expensive").show(5)


+-----+------------+
|price|is_expensive|
+-----+------------+
| 5953|       false|
| 5953|       false|
| 5956|       false|
| 5955|       false|
| 5955|       false|
+-----+------------+
only showing top 5 rows


In [14]:
# Promedios de cada seccion
df_clean.groupBy("airline").avg("price").show()
df_clean.groupBy("route").avg("duration").show()
df_clean.groupBy("airline").agg({"price": "min", "price": "max"}).show()
df_clean.groupBy("departure_time").count().show()


+---------+------------------+
|  airline|        avg(price)|
+---------+------------------+
|   Indigo| 5324.216303339517|
| SpiceJet| 6179.278881367218|
|Air_India| 23507.01911190229|
|  AirAsia|4091.0727419555224|
| GO_FIRST| 5652.007595045959|
|  Vistara| 30396.53630170735|
+---------+------------------+



+--------------------+------------------+
|               route|     avg(duration)|
+--------------------+------------------+
|Hyderabad → Banga...| 12.09331678643705|
|    Mumbai → Kolkata|12.836848115489666|
|    Mumbai → Chennai|12.665900287564627|
|  Mumbai → Hyderabad|13.263310412247066|
|  Mumbai → Bangalore|11.612022516178817|
|   Bangalore → Delhi|  9.77995566082195|
| Kolkata → Bangalore| 13.79294687524098|
|   Hyderabad → Delhi|10.829816602522587|
| Bangalore → Chennai|14.480207509137166|
|  Bangalore → Mumbai| 10.90507225639642|
|      Mumbai → Delhi|  9.81805726844943|
|  Hyderabad → Mumbai|11.962923295795918|
|   Kolkata → Chennai|14.774181563782903|
| Kolkata → Hyderabad|13.853107514948396|
|   Delhi → Bangalore| 10.35412503844018|
|      Delhi → Mumbai|10.367774213738123|
| Hyderabad → Chennai|13.293238468912078|
|Bangalore → Hyder...|14.162432783513621|
|     Kolkata → Delhi| 11.60498857561711|
|   Delhi → Hyderabad|12.518350118710492|
+--------------------+------------

+---------+----------+
|  airline|max(price)|
+---------+----------+
|   Indigo|     31952|
| SpiceJet|     34158|
|Air_India|     90970|
|  AirAsia|     31917|
| GO_FIRST|     32803|
|  Vistara|    123071|
+---------+----------+

+--------------+-----+
|departure_time|count|
+--------------+-----+
|       Evening|65102|
|       Morning|71146|
|    Late_Night| 1306|
|     Afternoon|47794|
| Early_Morning|66790|
|         Night|48015|
+--------------+-----+

